In [ ]:
%%capture
import sys
!{sys.executable} -m pip install -r requirements.txt
sys.path.append('../')

In [ ]:
# import needed libraries
import datetime
import glob
import itertools
import networkx
import numpy
import os
import pickle
import re
import requests
import tarfile
import shutil
import pandas as pd
import gffpandas.gffpandas as gffpd
import numpy as np
pd.set_option('display.max_columns', None)
import re

from collections import Counter
from functools import reduce
from rdflib import Graph, Namespace, URIRef, BNode, Literal
from rdflib.namespace import OWL, RDF, RDFS
from reactome2py import content
from tqdm import tqdm
from typing import Dict

from pkt_kg.utils import * 
from builds.ontology_cleaning import *

from typing import Tuple

In [ ]:
# directory to store resources
resource_data_location = '../resources/'

# directory to use for unprocessed data
unprocessed_data_location = '../resources/processed_data/unprocessed_data/'

# directory to use for processed data
processed_data_location = '../resources/processed_data/'

# directory to write ontology data to
ontology_data_location = '../resources/ontologies/'

# directory to write edges data to
edge_data_location = '../resources/edge_data/'

# directory to write node properties to
properties_location = '../resources/properties_data/'

# processed data url 
processed_url = 'https://storage.googleapis.com/pheknowlator/current_build/data/processed_data/'

# original data url 
original_url = 'https://storage.googleapis.com/pheknowlator/current_build/data/original_data/'

# owltools location
owltools_location = '../pkt_kg/libs/owltools'

***
### Utility functions

In [ ]:
# Get dbxrefs for all ontology classes' label
def gets_ontology_class_label(graph: Graph) -> Tuple:
    dbx_uris: Dict = dict()
    dbx = [x for x in graph if 'label' in str(x[1]).lower() if isinstance(x[0], URIRef)]
    for x in dbx:
        if str(x[2]).lower() in dbx_uris.keys(): dbx_uris[str(x[2]).lower()].append(str(x[0]))
        else: dbx_uris[str(x[2]).lower()] = [str(x[0])]
    dbx_type = {str(x[2]).lower(): 'DbXref' for x in dbx}

    ex_uris: Dict = dict()
    ex = [x for x in graph if 'exactmatch' in str(x[1]).lower() if isinstance([0], URIRef)]
    for x in ex:
        if str(x[2]).lower() in ex_uris.keys(): ex_uris[str(x[2]).lower()].append(str(x[0]))
        else: ex_uris[str(x[2]).lower()] = [str(x[0])]
    ex_type = {str(x[2]).lower(): 'ExactMatch' for x in ex}

    return {**dbx_uris, **ex_uris}, {**dbx_type, **ex_type}

In [ ]:
# Get dbxrefs for all ontology classes' label
def gets_ontology_class_synonym(graph: Graph) -> Tuple:
    dbx_uris: Dict = dict()
    dbx = [x for x in graph if 'synonym' in str(x[1]).lower() if isinstance(x[0], URIRef)]
    for x in dbx:
        if str(x[2]).lower() in dbx_uris.keys(): dbx_uris[str(x[2]).lower()].append(str(x[0]))
        else: dbx_uris[str(x[2]).lower()] = [str(x[0])]
    dbx_type = {str(x[2]).lower(): 'DbXref' for x in dbx}

    ex_uris: Dict = dict()
    ex = [x for x in graph if 'exactmatch' in str(x[1]).lower() if isinstance([0], URIRef)]
    for x in ex:
        if str(x[2]).lower() in ex_uris.keys(): ex_uris[str(x[2]).lower()].append(str(x[0]))
        else: ex_uris[str(x[2]).lower()] = [str(x[0])]
    ex_type = {str(x[2]).lower(): 'ExactMatch' for x in ex}

    return {**dbx_uris, **ex_uris}, {**dbx_type, **ex_type}

In [ ]:
data_downloader('https://www.mirbase.org/download/hsa.gff3', unprocessed_data_location)

miRBaseMap = gffpd.read_gff3(unprocessed_data_location + 'hsa.gff3')  
os.remove(unprocessed_data_location + 'hsa.gff3')

miRBaseMap = miRBaseMap.attributes_to_columns()
miRBaseMap = miRBaseMap[['attributes']]
miRBaseMap

miRBaseMap = miRBaseMap.attributes.str.split(';',expand=True)
# Keep only "ID" and "Name" columns
miRBaseMap = miRBaseMap[[2,0]]
# Remove substring "ID="
miRBaseMap[0] = miRBaseMap[0].str[3:]
# Remove substring "Name="
miRBaseMap[2] = miRBaseMap[2].str[5:]

***
# miRNA-mRNA

In [ ]:
maturemRNA_miRNA = pd.read_csv(
    edge_data_location + 'testmiRNA-mRNA.txt', sep='\t',
    names=['miRNA', 'ENTREZID', 'mRNA', 'miRDB score', 'Pubmed_id', 'Target site_number', 'miRNA_regulation',
           'Reporter_target gene/region', 'Reporter link element', 'Test_method_inter', 'Target gene mRNA_level',
           'Original description', 'Mutation_target region', 'Post mutation_method',
           'Original description_mutation_region', 'Target site_position', 'miRNA_regulation_site',
           'Reporter_target site', 'Reporter link element.1', 'Test_method_inter_site',
           'Original description_inter_site', 'Mutation_target site', 'Post mutation_method_site',
           'Original description_mutation_site', 'Additional note', 'tissue', 'category',
           'method', 'positive_negative', 'direct_indirect', 'up_down', 'condition', 'Support Type', 'Site Type',
           'UTR_start', 'UTR end', 'TargetScan context++ score', 'TargetScan context++ score percentile',
           'TargetScan weighted context++ score',
           'TargetScan weighted context++ score percentile', 'Predicted relative KD', 'U_mut_id', 'Mutationid', 'Refseq',
           'Mutationlocation', 'WTallele', 'Mutantallele', 'Targetsiteclass', 'Seed_mod', 'mRNAseq', 'miRseedseq',
           'Seedclass', 'WTconservation', 'Sample_id', 'wildtype_csp', 'mutant_csp', 'display_first', 'pita_ref',
           'pita_mut', 'pita_diff', 'SNP', 'diseases', 'distance', 'Source(s)', 'cell_line'], dtype=object)
maturemRNA_miRNA

### disease

In [ ]:
mondo_graph = Graph().parse(ontology_data_location + 'mondo_with_imports.owl')

mondo_label = gets_ontology_class_label(mondo_graph)[0]
mondo_dict = {str(k): {str(i).split('/')[-1] for i in v} for k, v in mondo_label.items()}
list({**mondo_dict}.items())[:5]

with open(unprocessed_data_location + 'DESC_Mondo_MAP.txt', 'w') as outfile:
    for k, v in {**mondo_dict}.items():
        outfile.write(str(k) + '\t' + str(v).replace('{','').replace('\'','').replace('}','') + '\n')

desc_mondo_map = pd.read_csv(unprocessed_data_location+'DESC_Mondo_MAP.txt', header=None, delimiter='\t')
desc_mondo_map[1] = desc_mondo_map[1].str.split(', ')
desc_mondo_map = desc_mondo_map.explode(1)

mondo_syn = gets_ontology_class_synonym(mondo_graph)[0]
mondo_dict = {str(k): {str(i).split('/')[-1] for i in v} for k, v in mondo_syn.items()}

with open(unprocessed_data_location + 'SYN_Mondo_MAP.txt', 'w') as outfile:
    for k, v in {**mondo_dict}.items():
        outfile.write(str(k) + '\t' + str(v).replace('{','').replace('\'','').replace('}','') + '\n')

syn_mondo_map = pd.read_csv(unprocessed_data_location+'SYN_Mondo_MAP.txt', header=None, delimiter='\t')
syn_mondo_map[1] = syn_mondo_map[1].str.split(', ')
syn_mondo_map = syn_mondo_map.explode(1)
desc_mondo_map = pd.concat([desc_mondo_map, syn_mondo_map], ignore_index=True)
desc_mondo_map

In [ ]:
maturemRNA_miRNA.diseases = maturemRNA_miRNA.diseases.str.lower()
maturemRNA_miRNA.diseases = maturemRNA_miRNA.diseases.str.replace("[ns]", ']')
maturemRNA_miRNA.diseases = maturemRNA_miRNA.diseases.str.replace("]", '|')
maturemRNA_miRNA.diseases = maturemRNA_miRNA.diseases.str.replace("[", '')
maturemRNA_miRNA.diseases = maturemRNA_miRNA.diseases.str.replace(", ", '|')
maturemRNA_miRNA.diseases = maturemRNA_miRNA.diseases.str.replace("||", '|')
maturemRNA_miRNA.diseases = maturemRNA_miRNA.diseases.replace("|||", np.nan)
maturemRNA_miRNA.diseases = maturemRNA_miRNA.diseases.str.replace('_', ' ')
maturemRNA_miRNA.diseases = maturemRNA_miRNA.diseases.str.replace('lung/sclc', 'small cell lung carcinoma')
maturemRNA_miRNA.diseases = maturemRNA_miRNA.diseases.replace(r'\(.*?\)', '', regex=True)
maturemRNA_miRNA.diseases = maturemRNA_miRNA.diseases.replace(r'\|$', '', regex=True)
maturemRNA_miRNA.diseases.unique()

In [ ]:
mondo_dict = dict(zip(desc_mondo_map[0], 'http://purl.obolibrary.org/obo/'+desc_mondo_map[1]))

def replace_with_mondo(substring):
    if pd.isna(substring):
        return np.nan
    else:
        return '|'.join([mondo_dict.get(part, part) for part in substring.split('|')])

maturemRNA_miRNA.diseases = [replace_with_mondo(item) for item in maturemRNA_miRNA.diseases]
maturemRNA_miRNA.diseases.unique()

### method

In [ ]:
maturemRNA_miRNA.method = maturemRNA_miRNA.method.str.lower()
maturemRNA_miRNA.method = maturemRNA_miRNA.method.str.replace("//", '|')
maturemRNA_miRNA.method = maturemRNA_miRNA.method.str.replace("/", '|')

maturemRNA_miRNA.method.unique()

In [ ]:
data_downloader('http://purl.obolibrary.org/obo/ncit.owl', ontology_data_location)

In [ ]:
ncit_graph = Graph().parse(ontology_data_location + 'ncit.owl')

ncit_label = gets_ontology_class_label(ncit_graph)[0]
ncit_dict = {str(k): {str(i).split('/')[-1] for i in v} for k, v in ncit_label.items()}
list({**ncit_dict}.items())[:5]

with open(unprocessed_data_location + 'DESC_ncit_MAP.txt', 'w') as outfile:
    for k, v in {**ncit_dict}.items():
        outfile.write(str(k) + '\t' + str(v).replace('{','').replace('\'','').replace('}','') + '\n')

desc_ncit_map = pd.read_csv(unprocessed_data_location+'DESC_ncit_MAP.txt', header=None, delimiter='\t')
desc_ncit_map[1] = desc_ncit_map[1].str.split(', ')
desc_ncit_map = desc_ncit_map.explode(1)

ncit_syn = gets_ontology_class_synonym(ncit_graph)[0]
ncit_dict = {str(k): {str(i).split('/')[-1] for i in v} for k, v in ncit_syn.items()}

with open(unprocessed_data_location + 'SYN_ncit_MAP.txt', 'w') as outfile:
    for k, v in {**ncit_dict}.items():
        outfile.write(str(k) + '\t' + str(v).replace('{','').replace('\'','').replace('}','') + '\n')

syn_ncit_map = pd.read_csv(unprocessed_data_location+'SYN_ncit_MAP.txt', header=None, delimiter='\t')
syn_ncit_map[1] = syn_ncit_map[1].str.split(', ')
syn_ncit_map = syn_ncit_map.explode(1)
desc_ncit_map = pd.concat([desc_ncit_map, syn_ncit_map], ignore_index=True)
desc_ncit_map

In [ ]:
ncit_dict = dict(zip(desc_ncit_map[0], 'http://purl.obolibrary.org/obo/'+desc_ncit_map[1]))

def replace_with_ncit(substring):
    if pd.isna(substring):
        return np.nan
    else:
        return '|'.join([ncit_dict.get(part, part) for part in substring.split('|')])

maturemRNA_miRNA.method = [replace_with_ncit(item) for item in maturemRNA_miRNA.method]
maturemRNA_miRNA.method.unique()

### tissue

In [ ]:
maturemRNA_miRNA.tissue = maturemRNA_miRNA.tissue.str.lower()
maturemRNA_miRNA.tissue = maturemRNA_miRNA.tissue.str.replace("/", '|')
maturemRNA_miRNA.tissue = maturemRNA_miRNA.tissue.str.replace("larva, whole", 'larva')

maturemRNA_miRNA.tissue.unique()

In [ ]:
uberon_graph = Graph().parse(ontology_data_location + 'ext_with_imports.owl')

uberon_label = gets_ontology_class_label(uberon_graph)[0]
uberon_dict = {str(k): {str(i).split('/')[-1] for i in v} for k, v in uberon_label.items()}
list({**uberon_dict}.items())[:5]

with open(unprocessed_data_location + 'DESC_Uberon_MAP.txt', 'w') as outfile:
    for k, v in {**uberon_dict}.items():
        outfile.write(str(k) + '\t' + str(v).replace('{','').replace('\'','').replace('}','') + '\n')

desc_uberon_map = pd.read_csv(unprocessed_data_location+'DESC_Uberon_MAP.txt', header=None, delimiter='\t')
desc_uberon_map[1] = desc_uberon_map[1].str.split(', ')
desc_uberon_map = desc_uberon_map.explode(1)

uberon_syn = gets_ontology_class_synonym(uberon_graph)[0]
uberon_dict = {str(k): {str(i).split('/')[-1] for i in v} for k, v in uberon_syn.items()}

with open(unprocessed_data_location + 'SYN_Uberon_MAP.txt', 'w') as outfile:
    for k, v in {**uberon_dict}.items():
        outfile.write(str(k) + '\t' + str(v).replace('{','').replace('\'','').replace('}','') + '\n')

syn_uberon_map = pd.read_csv(unprocessed_data_location+'SYN_Uberon_MAP.txt', header=None, delimiter='\t')
syn_uberon_map[1] = syn_uberon_map[1].str.split(', ')
syn_uberon_map = syn_uberon_map.explode(1)
desc_uberon_map = pd.concat([desc_uberon_map, syn_uberon_map], ignore_index=True)
desc_uberon_map

In [ ]:
uberon_dict = dict(zip(desc_uberon_map[0], 'http://purl.obolibrary.org/obo/'+desc_uberon_map[1]))

def replace_with_uberon(substring):
    if pd.isna(substring):
        return np.nan
    else:
        return '|'.join([uberon_dict.get(part, part) for part in substring.split('|')])

maturemRNA_miRNA.tissue = [replace_with_uberon(item) for item in maturemRNA_miRNA.tissue]
maturemRNA_miRNA.tissue.unique()

***
# miRNA-pseudogene
#### nota: da applicare a monte a dataframe miRNA_epiMod

In [ ]:
maturemiRNA_epiMod_class = pd.read_csv(
    edge_data_location + 'RmiRNA-epiModclass.txt', names=[
        'mir_acc','epi_modification','epi_regulator','experiment','expression','pmid','epi_target',
        'condition','detect','support','Source(s)'], sep='\t')
maturemiRNA_epiMod_class

#### epi target

In [ ]:
maturemiRNA_epiMod_class.epi_target = maturemiRNA_epiMod_class.epi_target.str.lower()
maturemiRNA_epiMod_class.epi_target = maturemiRNA_epiMod_class.epi_target.str.replace("/", '|')
maturemiRNA_epiMod_class.epi_target = maturemiRNA_epiMod_class.epi_target.str.replace(" cluster", '')
maturemiRNA_epiMod_class.epi_target = maturemiRNA_epiMod_class.epi_target.replace("\\\\\\\\N", np.nan)
maturemiRNA_epiMod_class.epi_target = maturemiRNA_epiMod_class.epi_target.replace(r'\(.*?\)', '', regex=True)
maturemiRNA_epiMod_class.epi_target = maturemiRNA_epiMod_class.epi_target.replace(r' \(.*?\)', '', regex=True)
maturemiRNA_epiMod_class.epi_target.unique()

In [ ]:
pro_graph = Graph().parse(ontology_data_location + 'pr_with_imports.owl')

pro_label = gets_ontology_class_label(pro_graph)[0]
pro_dict = {str(k): {str(i).split('/')[-1] for i in v} for k, v in pro_label.items()}
list({**pro_dict}.items())[:5]

with open(unprocessed_data_location + 'DESC_PRO_MAP.txt', 'w') as outfile:
    for k, v in {**pro_dict}.items():
        outfile.write(str(k) + '\t' + str(v).replace('{','').replace('\'','').replace('}','') + '\n')

desc_pro_map = pd.read_csv(unprocessed_data_location+'DESC_PRO_MAP.txt', header=None, delimiter='\t')
desc_pro_map[1] = desc_pro_map[1].str.split(', ')
desc_pro_map = desc_pro_map.explode(1)

pro_syn = gets_ontology_class_synonym(pro_graph)[0]
pro_dict = {str(k): {str(i).split('/')[-1] for i in v} for k, v in pro_syn.items()}

with open(unprocessed_data_location + 'SYN_PRO_MAP.txt', 'w') as outfile:
    for k, v in {**pro_dict}.items():
        outfile.write(str(k) + '\t' + str(v).replace('{','').replace('\'','').replace('}','') + '\n')

syn_pro_map = pd.read_csv(unprocessed_data_location+'SYN_PRO_MAP.txt', header=None, delimiter='\t')
syn_pro_map[1] = syn_pro_map[1].str.split(', ')
syn_pro_map = syn_pro_map.explode(1)
desc_pro_map = pd.concat([desc_pro_map, syn_pro_map], ignore_index=True)
desc_pro_map

pro_dict = dict(zip(desc_pro_map[0], 'http://purl.obolibrary.org/obo/'+desc_pro_map[1]))

def replace_with_pro(substring):
    if pd.isna(substring):
        return np.nan
    else:
        return '|'.join([pro_dict.get(part, part) for part in substring.split('|')])

maturemiRNA_epiMod_class.epi_target = [replace_with_pro(item) for item in maturemiRNA_epiMod_class.epi_target]
maturemiRNA_epiMod_class.epi_target.unique()

#### condition

In [ ]:
maturemiRNA_epiMod_class.condition.unique()

In [ ]:
maturemiRNA_epiMod_class.condition = maturemiRNA_epiMod_class.condition.str.replace('non-small-cell lung cancer', 'non-small cell lung carcinoma')
maturemiRNA_epiMod_class.condition = [replace_with_mondo(item) for item in maturemiRNA_epiMod_class.condition]
maturemiRNA_epiMod_class.condition.unique()

***
# miRNA-disease

In [ ]:
miRNA_disease=pd.read_csv(
    edge_data_location + 'testmiRNA-disease.txt', names=[0,4,5,'pmid_x','method','pmid_y','evidence',
                                                         'CancerSubtype','Cellline','ExperimentalDesign',
                                                         'adjPvalue','gene_name','refseq_id','SNP',
                                                         'distance',1,'Source(s)'], sep='\t')
miRNA_disease

#### method

In [ ]:
miRNA_disease['method'].unique()

In [ ]:
miRNA_disease.method = miRNA_disease.method.str.lower()
miRNA_disease.method = miRNA_disease.method.str.replace(' etc', '')
miRNA_disease.method = miRNA_disease.method.str.replace(', ', '|')
miRNA_disease.method = miRNA_disease.method.str.replace(' + ', '|')
miRNA_disease.method = [replace_with_ncit(item) for item in miRNA_disease.method]
miRNA_disease.method.unique()

#### disease

In [ ]:
miRNA_disease.CancerSubtype = miRNA_disease.CancerSubtype.str.lower()
miRNA_disease.CancerSubtype = [replace_with_mondo(item) for item in miRNA_disease.CancerSubtype]
miRNA_disease.CancerSubtype.unique()

#### cell line

In [ ]:
clo_graph = Graph().parse(ontology_data_location + 'clo_with_imports.owl')

clo_label = gets_ontology_class_label(clo_graph)[0]
clo_dict = {str(k): {str(i).split('/')[-1] for i in v} for k, v in clo_label.items()}
list({**clo_dict}.items())[:5]

with open(unprocessed_data_location + 'DESC_CLO_MAP.txt', 'w') as outfile:
    for k, v in {**clo_dict}.items():
        outfile.write(str(k) + '\t' + str(v).replace('{','').replace('\'','').replace('}','') + '\n')

desc_clo_map = pd.read_csv(unprocessed_data_location+'DESC_CLO_MAP.txt', header=None, delimiter='\t')
desc_clo_map[1] = desc_clo_map[1].str.split(', ')
desc_clo_map = desc_clo_map.explode(1)

clo_syn = gets_ontology_class_synonym(clo_graph)[0]
clo_dict = {str(k): {str(i).split('/')[-1] for i in v} for k, v in clo_syn.items()}

with open(unprocessed_data_location + 'SYN_CLO_MAP.txt', 'w') as outfile:
    for k, v in {**clo_dict}.items():
        outfile.write(str(k) + '\t' + str(v).replace('{','').replace('\'','').replace('}','') + '\n')

syn_clo_map = pd.read_csv(unprocessed_data_location+'SYN_CLO_MAP.txt', header=None, delimiter='\t')
syn_clo_map[1] = syn_clo_map[1].str.split(', ')
syn_clo_map = syn_clo_map.explode(1)
desc_clo_map = pd.concat([desc_clo_map, syn_clo_map], ignore_index=True)
desc_clo_map

In [ ]:
clo_dict = dict(zip(desc_clo_map[0], 'http://purl.obolibrary.org/obo/'+desc_clo_map[1]))

def replace_with_clo(substring):
    if pd.isna(substring):
        return np.nan
    else:
        return '|'.join([clo_dict.get(part, part) for part in substring.split('|')])

miRNA_disease.Cellline = miRNA_disease.Cellline.str.lower()
miRNA_disease.Cellline = miRNA_disease.Cellline.str.replace(',', ' cell|')
miRNA_disease.Cellline = miRNA_disease.Cellline + ' cell'
miRNA_disease.Cellline = [replace_with_clo(item) for item in miRNA_disease.Cellline]
miRNA_disease.Cellline.unique()

***
# miRNA-lncRNA

In [ ]:
miRNA_lncRNA=pd.read_csv(
    edge_data_location + 'testmiRNA-lncRNA.txt',
    names=['mir_id', '0_y', 'symbol', 'conservation', 'conservation_ortholog', 'conservation_species',
           'biological_context', 'context_detail', 'tissue/cell line', 'genome_variation', 'variation_detail',
           'epigenetic_modification', 'modification_detail', 'expression', 'expression_detail', 'regulator_type',
           'regulator', 'regulator_interaction', 'regulator_effect', 'target_type', 'target_effect',
           'experimental_method', 'molecular_function', 'biological_process', 'pathway', 'functional_mechanism',
           'clinical_detail', 'drug', 'description', 'pmid', 'Transcript', 'Mut_ID', 'COSMIC_ID', 'Chromosome',
           'Location', 'Ref_Allele', 'Mut_Allele', 'FuncClass', 'Alteration', 'Target_Site', 'Seed', 'SeedClass',
           'TargetScan_Site(0=No;1=Yes)', 'Sample_Name', 'Cancer_Class', 0, 'Source(s)'], sep='\t',dtype=object)
miRNA_lncRNA

#### disease

In [ ]:
miRNA_lncRNA.context_detail = [replace_with_mondo(item) for item in miRNA_lncRNA.context_detail]
miRNA_lncRNA.context_detail.unique()

#### tissue/cell line

In [ ]:
clo_dict2 = dict(zip(desc_clo_map[0].str.replace(' cell',''), 'http://purl.obolibrary.org/obo/'+desc_clo_map[1]))

def replace_with_clo2(substring):
    if pd.isna(substring):
        return np.nan
    else:
        return '|'.join([clo_dict2.get(part, part) for part in substring.split('|')])

miRNA_lncRNA['tissue/cell line'] = miRNA_lncRNA['tissue/cell line'].str.lower()
miRNA_lncRNA['tissue/cell line'] = miRNA_lncRNA['tissue/cell line'].str.replace(';', '|')
miRNA_lncRNA['tissue/cell line'] = [replace_with_clo(item) for item in miRNA_lncRNA['tissue/cell line']]
miRNA_lncRNA['tissue/cell line'] = [replace_with_clo2(item) for item in miRNA_lncRNA['tissue/cell line']]
miRNA_lncRNA['tissue/cell line'] = [replace_with_uberon(item) for item in miRNA_lncRNA['tissue/cell line']]
miRNA_lncRNA['tissue/cell line'].unique()

#### experimental method

In [ ]:
print(miRNA_lncRNA['experimental_method'].unique())
miRNA_lncRNA['experimental_method'] = miRNA_lncRNA['experimental_method'].str.replace('wetern', 'western')
miRNA_lncRNA['experimental_method'] = miRNA_lncRNA['experimental_method'].str.replace('assay assay', 'assay')
miRNA_lncRNA['experimental_method'] = miRNA_lncRNA['experimental_method'].str.lower()
miRNA_lncRNA['experimental_method'] = miRNA_lncRNA['experimental_method'].str.replace(';', '|')
miRNA_lncRNA['experimental_method'] = [replace_with_ncit(item) for item in miRNA_lncRNA['experimental_method']]

#### biological process

In [ ]:
go_graph = Graph().parse(ontology_data_location + 'go_with_imports.owl')

go_label = gets_ontology_class_label(go_graph)[0]
go_dict = {str(k): {str(i).split('/')[-1] for i in v} for k, v in go_label.items()}
list({**go_dict}.items())[:5]

with open(unprocessed_data_location + 'DESC_GO_MAP.txt', 'w') as outfile:
    for k, v in {**go_dict}.items():
        outfile.write(str(k) + '\t' + str(v).replace('{','').replace('\'','').replace('}','') + '\n')

desc_go_map = pd.read_csv(unprocessed_data_location+'DESC_GO_MAP.txt', header=None, delimiter='\t')
desc_go_map[1] = desc_go_map[1].str.split(', ')
desc_go_map = desc_go_map.explode(1)

go_syn = gets_ontology_class_synonym(go_graph)[0]
go_dict = {str(k): {str(i).split('/')[-1] for i in v} for k, v in go_syn.items()}

with open(unprocessed_data_location + 'SYN_GO_MAP.txt', 'w') as outfile:
    for k, v in {**go_dict}.items():
        outfile.write(str(k) + '\t' + str(v).replace('{','').replace('\'','').replace('}','') + '\n')

syn_go_map = pd.read_csv(unprocessed_data_location+'SYN_GO_MAP.txt', header=None, delimiter='\t')
syn_go_map[1] = syn_go_map[1].str.split(', ')
syn_go_map = syn_go_map.explode(1)
desc_go_map = pd.concat([desc_go_map, syn_go_map], ignore_index=True)
desc_go_map

In [ ]:
go_dict = dict(zip(desc_go_map[0], 'http://purl.obolibrary.org/obo/'+desc_go_map[1]))

def replace_with_go(substring):
    if pd.isna(substring):
        return np.nan
    else:
        return '|'.join([go_dict.get(part, part) for part in substring.split('|')])

In [ ]:
print(miRNA_lncRNA['biological_process'].unique())
miRNA_lncRNA['biological_process'] = miRNA_lncRNA['biological_process'].str.lower()
miRNA_lncRNA['biological_process'] = miRNA_lncRNA['biological_process'].str.replace(';', '|')
miRNA_lncRNA['biological_process'] = [replace_with_go(item) for item in miRNA_lncRNA['biological_process']]

#### pathway

In [ ]:
desc_pw_map = pd.read_csv(unprocessed_data_location+'DESC_PW_MAP.txt', header=None, delimiter='\t')
desc_pw_map[1] = desc_pw_map[1].str.split(', ')
desc_pw_map = desc_pw_map.explode(1)

syn_pw_map = pd.read_csv(unprocessed_data_location+'SYN_PW_MAP.txt', header=None, delimiter='\t')
syn_pw_map[1] = syn_pw_map[1].str.split(', ')
syn_pw_map = syn_pw_map.explode(1)
desc_pw_map = pd.concat([desc_pw_map, syn_pw_map], ignore_index=True)
desc_pw_map

pw_dict = dict(zip(desc_pw_map[0], 'http://purl.obolibrary.org/obo/'+desc_pw_map[1]))

def replace_with_pw(substring):
    if pd.isna(substring):
        return np.nan
    else:
        return '|'.join([pw_dict.get(part, part) for part in substring.split('|')])

In [ ]:
print(miRNA_lncRNA['pathway'].unique())
miRNA_lncRNA['pathway'] = miRNA_lncRNA['pathway'].str.lower()
miRNA_lncRNA['pathway'] = miRNA_lncRNA['pathway'].str.replace(';', '|')
miRNA_lncRNA['pathway'] = miRNA_lncRNA['pathway'].str.replace('nf-??b', 'nuclear factor kappa B')
miRNA_lncRNA['pathway'] = miRNA_lncRNA['pathway'].str.replace('/??-catenin', '')
miRNA_lncRNA['pathway'] = miRNA_lncRNA['pathway'].str.replace('pi3k/akt ', '')
miRNA_lncRNA['pathway'] = [replace_with_pw(item) for item in miRNA_lncRNA['pathway']]

#### drug

In [ ]:
DrugBank = pd.read_csv(unprocessed_data_location + 'drugbank vocabulary.csv') 
DrugBank['Common name'] = DrugBank['Common name'].str.lower()
DrugBank

In [ ]:
drugbank_dict = dict(zip(DrugBank['Common name'], 'https://go.drugbank.com/drugs/'+DrugBank['DrugBank ID']))
def replace_with_drugbank(substring):
    if pd.isna(substring):
        return np.nan
    else:
        return '|'.join([drugbank_dict.get(part, part) for part in substring.split('|')])

print(miRNA_lncRNA['drug'].unique())
miRNA_lncRNA['drug'] = miRNA_lncRNA['drug'].str.lower()
miRNA_lncRNA['drug'] = miRNA_lncRNA['drug'].str.replace(';', '|')

miRNA_lncRNA['drug'] = [replace_with_drugbank(item) for item in miRNA_lncRNA['drug']]

#### chromosome

In [ ]:
desc_ncit_map = pd.read_csv(unprocessed_data_location+'DESC_ncit_MAP.txt', header=None, delimiter='\t')
desc_ncit_map[1] = desc_ncit_map[1].str.split(', ')
desc_ncit_map = desc_ncit_map.explode(1)

syn_ncit_map = pd.read_csv(unprocessed_data_location+'SYN_ncit_MAP.txt', header=None, delimiter='\t')
syn_ncit_map[1] = syn_ncit_map[1].str.split(', ')
syn_ncit_map = syn_ncit_map.explode(1)
desc_ncit_map = pd.concat([desc_ncit_map, syn_ncit_map], ignore_index=True)
desc_ncit_map

ncit_dict = dict(zip(desc_ncit_map[0], 'http://purl.obolibrary.org/obo/'+desc_ncit_map[1]))

def replace_with_ncit(substring):
    if pd.isna(substring):
        return np.nan
    else:
        return '|'.join([ncit_dict.get(part, part) for part in substring.split('|')])

In [ ]:
print(miRNA_lncRNA['Chromosome'].unique())
miRNA_lncRNA['Chromosome'] = miRNA_lncRNA['Chromosome'].str.lower()
miRNA_lncRNA['Chromosome'] = [replace_with_ncit(item) for item in miRNA_lncRNA['Chromosome']]

#### cancer class

In [ ]:
print(miRNA_lncRNA['Cancer_Class'].unique())
miRNA_lncRNA['Cancer_Class'] = miRNA_lncRNA['Cancer_Class'].str.lower()
miRNA_lncRNA['Cancer_Class'] = miRNA_lncRNA['Cancer_Class'].str.replace("[ns]", ']')
miRNA_lncRNA['Cancer_Class'] = miRNA_lncRNA['Cancer_Class'].str.replace("]", '|')
miRNA_lncRNA['Cancer_Class'] = miRNA_lncRNA['Cancer_Class'].str.replace("[", '')
miRNA_lncRNA['Cancer_Class'] = miRNA_lncRNA['Cancer_Class'].str.replace(", ", '|')
miRNA_lncRNA['Cancer_Class'] = miRNA_lncRNA['Cancer_Class'].str.replace("||", '|')
miRNA_lncRNA['Cancer_Class'] = miRNA_lncRNA['Cancer_Class'].str.replace('_', ' ')
miRNA_lncRNA['Cancer_Class'] = miRNA_lncRNA['Cancer_Class'].str.replace('lung/sclc', 'small cell lung carcinoma')
miRNA_lncRNA['Cancer_Class'] = miRNA_lncRNA['Cancer_Class'].replace(r'\(.*?\)', '', regex=True)
miRNA_lncRNA['Cancer_Class'] = miRNA_lncRNA['Cancer_Class'].replace(r'\|$', '', regex=True)
miRNA_lncRNA['Cancer_Class'] = [replace_with_mondo(item) for item in miRNA_lncRNA['Cancer_Class']]

***
# miRNA-SNP

In [ ]:
miRNA_variant=pd.read_csv(
    edge_data_location + 'testmiRNA-variant.txt',
    names=['rsid','gnomAD_MAF','MIRNA_Name','MIRNA_Acc','MIRNA_Domain','Precursor_Pos','Predicted_Motif','gene_name','refseq_id','diseases','distance','experimentally_confirmed','Source(s)'], sep='\t',dtype=object)
miRNA_variant['Source(s)'] = miRNA_variant['Source(s)'].str.replace('nan\||\|nan', '', regex=True)
miRNA_variant

#### disease

In [ ]:
miRNA_variant['diseases'] = miRNA_variant['diseases'].str.lower()
miRNA_variant['diseases'] = miRNA_variant['diseases'].str.replace(", ", '|')
miRNA_variant['diseases'] = [replace_with_mondo(item) for item in miRNA_variant['diseases']]
miRNA_variant['diseases'].unique()

# lncRNA-gene

In [ ]:
lncRNA_gene=pd.read_csv(
    edge_data_location + 'testlncRNA-gene.txt', names=['transcript_id',
                                                       'conservation', 'conservation_ortholog',
                                                       'conservation_species', 'biological_context',
                                                       'context_detail',
       'tissue/cell line', 'genome_variation', 'variation_detail',
       'epigenetic_modification', 'modification_detail', 'expression',
       'expression_detail', 'regulator_type', 'regulator',
       'regulator_interaction', 'regulator_effect', 'target_interaction',
       'target_effect', 'experimental_method', 'molecular_function',
       'biological_process', 'pathway', 'functional_mechanism',
       'clinical_detail', 'drug', 'description', 'pmid', '0_y_x', '3_x_x',
       '0_y_y', '3_x_y', 'Source(s)'], sep='\t')

lncRNA_gene = lncRNA_gene.drop(columns=['3_x_x','3_x_y'])
lncRNA_gene

In [ ]:
#### disease

lncRNA_gene.context_detail = [replace_with_mondo(item) for item in lncRNA_gene.context_detail]

#### tissue/cell line

print(lncRNA_gene['tissue/cell line'].unique())
lncRNA_gene['tissue/cell line'] = lncRNA_gene['tissue/cell line'].str.lower()
lncRNA_gene['tissue/cell line'] = lncRNA_gene['tissue/cell line'].str.replace(';', '|')
lncRNA_gene['tissue/cell line'] = [replace_with_clo(item) for item in lncRNA_gene['tissue/cell line']]
lncRNA_gene['tissue/cell line'] = [replace_with_clo2(item) for item in lncRNA_gene['tissue/cell line']]
lncRNA_gene['tissue/cell line'] = [replace_with_uberon(item) for item in lncRNA_gene['tissue/cell line']]

#### experimental method

print(lncRNA_gene['experimental_method'].unique())
lncRNA_gene['experimental_method'] = lncRNA_gene['experimental_method'].str.replace('wetern', 'western')
lncRNA_gene['experimental_method'] = lncRNA_gene['experimental_method'].str.replace('assay assay', 'assay')
lncRNA_gene['experimental_method'] = lncRNA_gene['experimental_method'].str.lower()
lncRNA_gene['experimental_method'] = lncRNA_gene['experimental_method'].str.replace(';', '|')
lncRNA_gene['experimental_method'] = [replace_with_ncit(item) for item in lncRNA_gene['experimental_method']]

#### biological process

print(lncRNA_gene['biological_process'].unique())
lncRNA_gene['biological_process'] = lncRNA_gene['biological_process'].str.lower()
lncRNA_gene['biological_process'] = lncRNA_gene['biological_process'].str.replace(';', '|')
lncRNA_gene['biological_process'] = [replace_with_go(item) for item in lncRNA_gene['biological_process']]

#### pathway

print(lncRNA_gene['pathway'].unique())
lncRNA_gene['pathway'] = lncRNA_gene['pathway'].str.lower()
lncRNA_gene['pathway'] = lncRNA_gene['pathway'].str.replace(';', '|')
lncRNA_gene['pathway'] = lncRNA_gene['pathway'].str.replace('nf-??b', 'nuclear factor kappa B')
lncRNA_gene['pathway'] = lncRNA_gene['pathway'].str.replace('/??-catenin', '')
lncRNA_gene['pathway'] = lncRNA_gene['pathway'].str.replace('pi3k/akt ', '')
lncRNA_gene['pathway'] = [replace_with_pw(item) for item in lncRNA_gene['pathway']]

#### drug

print(lncRNA_gene['drug'].unique())
lncRNA_gene['drug'] = lncRNA_gene['drug'].str.lower()
lncRNA_gene['drug'] = lncRNA_gene['drug'].str.replace(';', '|')
lncRNA_gene['drug'] = [replace_with_drugbank(item) for item in lncRNA_gene['drug']]

# lncRNA-disease

In [ ]:
lncRNA_disease = pd.read_csv(
    edge_data_location + 'testlncRNA-disease.txt', names=['0_y', 'ncRNA Symbol', 'Sample', 'Dysfunction Pattern',
       'Validated Method', 'Description', 'PubMed ID', 'function description',
       'gene_id', 'transcript_id', 'conservation', 'conservation_ortholog',
       'conservation_species', 'biological_context', 'genome_variation',
       'variation_detail', 'epigenetic_modification', 'modification_detail',
       'expression', 'expression_detail', 'regulator_type', 'regulator',
       'regulator_interaction', 'regulator_effect', 'target_type', 'target',
       'target_interaction', 'target_effect', 'experimental_method',
       'molecular_function', 'biological_process', 'pathway',
       'functional_mechanism', 'clinical_detail', 'drug', 'description',
       'desc', 'Gene ID', 'Variant Name', 'dbSNP ID', 'COSMIC Mutation ID',
       'COSMIC Variation Effect', 'ClinVar Allele ID',
       'ClinVar Variation Effect', 'ClinVar Disease Name', 'GWAS Trait', '3_x',
       'Source(s)'], sep='\t', dtype=object)
lncRNA_disease

#### Dysfunction pattern

In [ ]:
lncRNA_disease.loc[(lncRNA_disease["Dysfunction Pattern"].notna()) &
                   (lncRNA_disease["expression_detail"].notna()),
                   ["Dysfunction Pattern"]] = lncRNA_disease['Dysfunction Pattern'] + '|' + lncRNA_disease['expression_detail']
lncRNA_disease = lncRNA_disease.drop(columns=['expression_detail'])

mirna_dict = dict(zip(miRBaseMap[2], miRBaseMap[0]))

def replace_with_mirbase(substring):
    if pd.isna(substring):
        return np.nan
    else:
        return '|'.join([mirna_dict.get(part, part) for part in substring.split('|')])

def extract_pattern(value):
    if pd.notna(value) and '[' in value and ']' in value:
        return pd.Series([value.split('[')[1]])
    else:
        return pd.Series([value])

lncRNA_disease['Dysfunction Pattern'] = lncRNA_disease['Dysfunction Pattern'].str.replace('/', '+')
lncRNA_disease['Dysfunction Pattern'] = lncRNA_disease['Dysfunction Pattern'].apply(extract_pattern)
lncRNA_disease['Dysfunction Pattern'] = lncRNA_disease['Dysfunction Pattern'].squeeze()

lncRNA_disease['Dysfunction Pattern'] = [replace_with_mirbase(item) for item in lncRNA_disease['Dysfunction Pattern']]
lncRNA_disease['Dysfunction Pattern'] = lncRNA_disease['Dysfunction Pattern'].str.replace('miR', 'hsa-miR')
lncRNA_disease['Dysfunction Pattern'] = [replace_with_mirbase(item) for item in lncRNA_disease['Dysfunction Pattern']]

lncRNA_disease['Dysfunction Pattern'] = lncRNA_disease['Dysfunction Pattern'].str.replace('Down-regulated','http://purl.obolibrary.org/obo/OMIT_0016265')
lncRNA_disease['Dysfunction Pattern'] = lncRNA_disease['Dysfunction Pattern'].str.replace('Up-regulated','http://purl.obolibrary.org/obo/OMIT_0016489')
lncRNA_disease['Dysfunction Pattern'] = lncRNA_disease['Dysfunction Pattern'].str.replace('Differentially expressed','http://purl.obolibrary.org/obo/OBI_0002584')
lncRNA_disease['Dysfunction Pattern'] = lncRNA_disease['Dysfunction Pattern'].str.replace('Mutation','http://purl.obolibrary.org/obo/OMIT_0010192')

lncRNA_disease['Dysfunction Pattern'] = lncRNA_disease['Dysfunction Pattern'].str.replace(']', '')
lncRNA_disease['Dysfunction Pattern'] = lncRNA_disease['Dysfunction Pattern'].str.replace('regulation', 'http://purl.obolibrary.org/obo/GO_0010468')

#### Validated Method

In [ ]:
lncRNA_disease.loc[(lncRNA_disease["Validated Method"].notna()) &
                   (lncRNA_disease["experimental_method"].notna()),
                   ["Validated Method"]] = lncRNA_disease['Validated Method'] + '|' + lncRNA_disease['experimental_method']
lncRNA_disease = lncRNA_disease.drop(columns=['experimental_method'])

lncRNA_disease['Validated Method'] = lncRNA_disease['Validated Method'].str.replace('wetern', 'western')
lncRNA_disease['Validated Method'] = lncRNA_disease['Validated Method'].str.replace('assay assay', 'assay')
lncRNA_disease['Validated Method'] = lncRNA_disease['Validated Method'].str.lower()
lncRNA_disease['Validated Method'] = lncRNA_disease['Validated Method'].str.replace('//', '|')
lncRNA_disease['Validated Method'] = lncRNA_disease['Validated Method'].str.replace(', ', '|')
lncRNA_disease['Validated Method'] = [replace_with_ncit(item) for item in lncRNA_disease['Validated Method']]

#### sample

In [ ]:
uberon_dict2 = dict(zip(desc_uberon_map[0].str.replace(' tissue',''), 'http://purl.obolibrary.org/obo/'+desc_uberon_map[1]))

def replace_with_uberon2(substring):
    if pd.isna(substring):
        return np.nan
    else:
        return '|'.join([uberon_dict2.get(part, part) for part in substring.split('|')])
    
lncRNA_disease.Sample = lncRNA_disease.Sample.str.replace(';', '|')
lncRNA_disease.Sample = lncRNA_disease.Sample.str.replace(',', '|')
lncRNA_disease.Sample = lncRNA_disease.Sample.str.replace('(', '')
lncRNA_disease.Sample = lncRNA_disease.Sample.str.replace(')', '')
lncRNA_disease.Sample = lncRNA_disease.Sample.str.replace(' as well as ', '|')
lncRNA_disease.Sample = lncRNA_disease.Sample.str.replace(' and ', '|')
lncRNA_disease.Sample = lncRNA_disease.Sample.str.replace(' |', '|')
lncRNA_disease.Sample = lncRNA_disease.Sample.str.replace('| ', '|')
lncRNA_disease.Sample = lncRNA_disease.Sample.str.replace(' | ', '|')
lncRNA_disease.Sample = lncRNA_disease.Sample.str.replace(', ', '|')
lncRNA_disease.Sample = [replace_with_clo(item) for item in lncRNA_disease.Sample]
lncRNA_disease.Sample = [replace_with_clo2(item) for item in lncRNA_disease.Sample]
lncRNA_disease.Sample = [replace_with_uberon(item) for item in lncRNA_disease.Sample]
lncRNA_disease.Sample = [replace_with_uberon2(item) for item in lncRNA_disease.Sample]
lncRNA_disease.Sample.unique()

#### target

In [ ]:
lncRNA_disease['target'] = lncRNA_disease['target'].str.replace('miR', 'hsa-miR')
lncRNA_disease['target'] = [replace_with_mirbase(item) for item in lncRNA_disease['target']]
lncRNA_disease['target'] = lncRNA_disease['target'].str.replace(';', '|').str.lower()
lncRNA_disease['target'] = [replace_with_pro(item) for item in lncRNA_disease['target']]
lncRNA_disease['target'].unique()

#### ClinVar Variation Effect

In [ ]:
lncRNA_disease['ClinVar Variation Effect'] = lncRNA_disease['ClinVar Variation Effect'].str.replace('; ', '|').str.lower()
lncRNA_disease['ClinVar Variation Effect'] = [replace_with_mondo(item) for item in lncRNA_disease['ClinVar Variation Effect']]
lncRNA_disease['ClinVar Variation Effect'] = lncRNA_disease['ClinVar Variation Effect'].str.replace(r's$', '', regex=True)
lncRNA_disease['ClinVar Variation Effect'] = [replace_with_mondo(item) for item in lncRNA_disease['ClinVar Variation Effect']]
lncRNA_disease['ClinVar Variation Effect'].unique()

#### GWAS Trait

In [ ]:
lncRNA_disease['GWAS Trait'] = lncRNA_disease['GWAS Trait'].str.replace('; ', '|')
lncRNA_disease['GWAS Trait'] = lncRNA_disease['GWAS Trait'].str.replace('; ', '|').str.lower()
lncRNA_disease['GWAS Trait'] = [replace_with_mondo(item) for item in lncRNA_disease['GWAS Trait']]
lncRNA_disease['GWAS Trait'] = lncRNA_disease['GWAS Trait'].str.replace(r's$', '', regex=True)
lncRNA_disease['GWAS Trait'] = [replace_with_mondo(item) for item in lncRNA_disease['GWAS Trait']]

lncRNA_disease['GWAS Trait'] = [replace_with_ncit(item) for item in lncRNA_disease['GWAS Trait']]
lncRNA_disease['GWAS Trait'].unique()

In [ ]:
#### biological process

lncRNA_disease['biological_process'] = lncRNA_disease['biological_process'].str.lower()
lncRNA_disease['biological_process'] = lncRNA_disease['biological_process'].str.replace(';', '|')
lncRNA_disease['biological_process'] = [replace_with_go(item) for item in lncRNA_disease['biological_process']]

#### pathway

print(lncRNA_disease['pathway'].unique())
lncRNA_disease['pathway'] = lncRNA_disease['pathway'].str.lower()
lncRNA_disease['pathway'] = lncRNA_disease['pathway'].str.replace(';', '|')
lncRNA_disease['pathway'] = lncRNA_disease['pathway'].str.replace('nf-??b', 'nuclear factor kappa B')
lncRNA_disease['pathway'] = lncRNA_disease['pathway'].str.replace('/??-catenin', '')
lncRNA_disease['pathway'] = lncRNA_disease['pathway'].str.replace('pi3k/akt ', '')
lncRNA_disease['pathway'] = [replace_with_pw(item) for item in lncRNA_disease['pathway']]

#### drug

print(lncRNA_disease['drug'].unique())
lncRNA_disease['drug'] = lncRNA_disease['drug'].str.lower()
lncRNA_disease['drug'] = lncRNA_disease['drug'].str.replace(';', '|')
lncRNA_disease['drug'] = [replace_with_drugbank(item) for item in lncRNA_disease['drug']]

#### regulator
lncRNA_disease['regulator'] = lncRNA_disease['regulator'].str.replace(';', '|').str.lower()
lncRNA_disease['regulator'] = [replace_with_pro(item) for item in lncRNA_disease['regulator']]

# circRNA-disease

In [ ]:
circRNA_disease = pd.read_csv(
    edge_data_location + 'testcircRNA-disease.txt', names=['0_y','Sample',
                                                          'Dysfunction Pattern','Validated Method','Description',
                                                          'PubMed ID','Source(s)',1], sep='\t', dtype=object)
circRNA_disease

In [ ]:
#### Dysfunction pattern

circRNA_disease['Dysfunction Pattern'] = circRNA_disease['Dysfunction Pattern'].apply(extract_pattern)
circRNA_disease['Dysfunction Pattern'] = circRNA_disease['Dysfunction Pattern'].squeeze()

circRNA_disease['Dysfunction Pattern'] = circRNA_disease['Dysfunction Pattern'].str.replace('down-regulated','http://purl.obolibrary.org/obo/OMIT_0016265')
circRNA_disease['Dysfunction Pattern'] = circRNA_disease['Dysfunction Pattern'].str.replace('up-regulated','http://purl.obolibrary.org/obo/OMIT_0016489')
circRNA_disease['Dysfunction Pattern'] = circRNA_disease['Dysfunction Pattern'].str.replace('differentially expressed','http://purl.obolibrary.org/obo/OBI_0002584')

circRNA_disease['Dysfunction Pattern'] = circRNA_disease['Dysfunction Pattern'].str.replace(']', '')
circRNA_disease['Dysfunction Pattern'] = circRNA_disease['Dysfunction Pattern'].str.replace('regulation', 'http://purl.obolibrary.org/obo/GO_0010468')

#### Validated Method

circRNA_disease['Validated Method'] = circRNA_disease['Validated Method'].str.replace('wetern', 'western')
circRNA_disease['Validated Method'] = circRNA_disease['Validated Method'].str.replace('assay assay', 'assay')
circRNA_disease['Validated Method'] = circRNA_disease['Validated Method'].str.lower()
circRNA_disease['Validated Method'] = circRNA_disease['Validated Method'].str.replace('//', '|')
circRNA_disease['Validated Method'] = circRNA_disease['Validated Method'].str.replace(', ', '|')
circRNA_disease['Validated Method'] = [replace_with_ncit(item) for item in circRNA_disease['Validated Method']]

#### sample

circRNA_disease.Sample = circRNA_disease.Sample.str.replace(';', '|')
circRNA_disease.Sample = circRNA_disease.Sample.str.replace(',', '|')
circRNA_disease.Sample = circRNA_disease.Sample.str.replace('(', '')
circRNA_disease.Sample = circRNA_disease.Sample.str.replace(')', '')
circRNA_disease.Sample = circRNA_disease.Sample.str.replace(' as well as ', '|')
circRNA_disease.Sample = circRNA_disease.Sample.str.replace(' and ', '|')
circRNA_disease.Sample = circRNA_disease.Sample.str.replace(' |', '|')
circRNA_disease.Sample = circRNA_disease.Sample.str.replace('| ', '|')
circRNA_disease.Sample = circRNA_disease.Sample.str.replace(' | ', '|')
circRNA_disease.Sample = circRNA_disease.Sample.str.replace(', ', '|')
circRNA_disease.Sample = [replace_with_clo(item) for item in circRNA_disease.Sample]
circRNA_disease.Sample = [replace_with_clo2(item) for item in circRNA_disease.Sample]
circRNA_disease.Sample = [replace_with_uberon(item) for item in circRNA_disease.Sample]
circRNA_disease.Sample = [replace_with_uberon2(item) for item in circRNA_disease.Sample]
circRNA_disease.Sample.unique()

# lncRNA-chemical

In [ ]:
lncRNA_chemical = pd.read_csv(
    edge_data_location + 'testlncRNA-chemical.txt', names=[1,'transcript_id','conservation',
                                                           'conservation_ortholog','conservation_species',
                                                           'biological_context','context_detail',
                                                           'tissue/cell line','genome_variation',
                                                           'variation_detail','epigenetic_modification',
                                                           'modification_detail','expression','expression_detail',
                                                           'regulator_type','regulator','regulator_interaction',
                                                           'regulator_effect','target_type','target',
                                                           'target_interaction','target_effect',
                                                           'experimental_method','molecular_function',
                                                           'biological_process','pathway','functional_mechanism',
                                                           'clinical_detail','description','pmid','0_y',
                                                           'Source(s)'], sep='\t', dtype=object)
lncRNA_chemical

In [ ]:
#### regulator

lncRNA_chemical['regulator'] = lncRNA_chemical['regulator'].str.replace(';', '|').str.lower()
lncRNA_chemical['regulator'] = [replace_with_pro(item) for item in lncRNA_chemical['regulator']]

#### target

lncRNA_chemical['target'] = lncRNA_chemical['target'].str.replace('miR', 'hsa-miR')
lncRNA_chemical['target'] = [replace_with_mirbase(item) for item in lncRNA_chemical['target']]
lncRNA_chemical['target'] = lncRNA_chemical['target'].str.replace(';', '|').str.lower()
lncRNA_chemical['target'] = [replace_with_pro(item) for item in lncRNA_chemical['target']]

#### disease

lncRNA_chemical.context_detail = [replace_with_mondo(item) for item in lncRNA_chemical.context_detail]

#### tissue/cell line

lncRNA_chemical['tissue/cell line'] = lncRNA_chemical['tissue/cell line'].str.lower()
lncRNA_chemical['tissue/cell line'] = lncRNA_chemical['tissue/cell line'].str.replace(';', '|')
lncRNA_chemical['tissue/cell line'] = [replace_with_clo(item) for item in lncRNA_chemical['tissue/cell line']]
lncRNA_chemical['tissue/cell line'] = [replace_with_clo2(item) for item in lncRNA_chemical['tissue/cell line']]
lncRNA_chemical['tissue/cell line'] = [replace_with_uberon(item) for item in lncRNA_chemical['tissue/cell line']]

#### experimental method

lncRNA_chemical['experimental_method'] = lncRNA_chemical['experimental_method'].str.replace('wetern', 'western')
lncRNA_chemical['experimental_method'] = lncRNA_chemical['experimental_method'].str.replace('assay assay', 'assay')
lncRNA_chemical['experimental_method'] = lncRNA_chemical['experimental_method'].str.lower()
lncRNA_chemical['experimental_method'] = lncRNA_chemical['experimental_method'].str.replace(';', '|')
lncRNA_chemical['experimental_method'] = [replace_with_ncit(item) for item in lncRNA_chemical['experimental_method']]

#### biological process

lncRNA_chemical['biological_process'] = lncRNA_chemical['biological_process'].str.lower()
lncRNA_chemical['biological_process'] = lncRNA_chemical['biological_process'].str.replace(';', '|')
lncRNA_chemical['biological_process'] = [replace_with_go(item) for item in lncRNA_chemical['biological_process']]

#### pathway

lncRNA_chemical['pathway'] = lncRNA_chemical['pathway'].str.lower()
lncRNA_chemical['pathway'] = lncRNA_chemical['pathway'].str.replace(';', '|')
lncRNA_chemical['pathway'] = lncRNA_chemical['pathway'].str.replace('nf-??b', 'nuclear factor kappa B')
lncRNA_chemical['pathway'] = lncRNA_chemical['pathway'].str.replace('/??-catenin', '')
lncRNA_chemical['pathway'] = lncRNA_chemical['pathway'].str.replace('pi3k/akt ', '')
lncRNA_chemical['pathway'] = [replace_with_pw(item) for item in lncRNA_chemical['pathway']]

# lncRNA-protein

In [ ]:
lncRNA_protein=pd.read_csv(
    edge_data_location + 'testlncRNA-protein.txt', names=['Transcript ID', 'Cell Line', 'Start', 'End', 'conservation',
       'conservation_ortholog', 'conservation_species', 'biological_context',
       'context_detail', 'tissue/cell line', 'genome_variation',
       'variation_detail', 'epigenetic_modification', 'modification_detail',
       'expression', 'expression_detail', 'regulator_type', 'regulator',
       'regulator_interaction', 'regulator_effect', 'target_type',
       'target_interaction', 'target_effect', 'experimental_method',
       'molecular_function', 'biological_process', 'pathway',
       'functional_mechanism', 'clinical_detail', 'drug', 'description',
       'pmid', '0_y', '1_y', 'Source(s)'], sep='\t', dtype=object)
lncRNA_protein

In [ ]:
#### disease

lncRNA_protein.context_detail = [replace_with_mondo(item) for item in lncRNA_protein.context_detail]

#### tissue/cell line

lncRNA_protein['tissue/cell line'] = lncRNA_protein['tissue/cell line'].str.lower()
lncRNA_protein['tissue/cell line'] = lncRNA_protein['tissue/cell line'].str.replace(';', '|')
lncRNA_protein['tissue/cell line'] = [replace_with_clo(item) for item in lncRNA_protein['tissue/cell line']]
lncRNA_protein['tissue/cell line'] = [replace_with_clo2(item) for item in lncRNA_protein['tissue/cell line']]
lncRNA_protein['tissue/cell line'] = [replace_with_uberon(item) for item in lncRNA_protein['tissue/cell line']]

#### experimental method

lncRNA_protein['experimental_method'] = lncRNA_protein['experimental_method'].str.replace('wetern', 'western')
lncRNA_protein['experimental_method'] = lncRNA_protein['experimental_method'].str.replace('assay assay', 'assay')
lncRNA_protein['experimental_method'] = lncRNA_protein['experimental_method'].str.lower()
lncRNA_protein['experimental_method'] = lncRNA_protein['experimental_method'].str.replace(';', '|')
lncRNA_protein['experimental_method'] = [replace_with_ncit(item) for item in lncRNA_protein['experimental_method']]

#### biological process

lncRNA_protein['biological_process'] = lncRNA_protein['biological_process'].str.lower()
lncRNA_protein['biological_process'] = lncRNA_protein['biological_process'].str.replace(';', '|')
lncRNA_protein['biological_process'] = [replace_with_go(item) for item in lncRNA_protein['biological_process']]

#### pathway

lncRNA_protein['pathway'] = lncRNA_protein['pathway'].str.lower()
lncRNA_protein['pathway'] = lncRNA_protein['pathway'].str.replace(';', '|')
lncRNA_protein['pathway'] = lncRNA_protein['pathway'].str.replace('nf-??b', 'nuclear factor kappa B')
lncRNA_protein['pathway'] = lncRNA_protein['pathway'].str.replace('/??-catenin', '')
lncRNA_protein['pathway'] = lncRNA_protein['pathway'].str.replace('pi3k/akt ', '')
lncRNA_protein['pathway'] = [replace_with_pw(item) for item in lncRNA_protein['pathway']]

#### drug

lncRNA_protein['drug'] = lncRNA_protein['drug'].str.lower()
lncRNA_protein['drug'] = lncRNA_protein['drug'].str.replace(';', '|')

lncRNA_protein['drug'] = [replace_with_drugbank(item) for item in lncRNA_protein['drug']]

# lncRNA-GOCC

In [ ]:
lncRNA_gocc = pd.read_csv(
    edge_data_location + 'testlncRNA-gocc.txt', names=['Data Source', 'Value', '0_y', 'gocc', 'Source(s)'],
    sep='\t', dtype=object)
lncRNA_gocc

In [ ]:
lncRNA_gocc['Data Source'] = lncRNA_gocc['Data Source'].str.replace('.', ' ').str.lower()
lncRNA_gocc['Data Source'] = [replace_with_clo(item) for item in lncRNA_gocc['Data Source']]
lncRNA_gocc['Data Source'] = [replace_with_clo2(item) for item in lncRNA_gocc['Data Source']]

lncRNA_gocc['Data Source'].unique()

# lncRNA-gobp

In [ ]:
lncRNA_gobp2=pd.read_csv(
    edge_data_location + 'testlncRNA-gobp.txt', names=['transcript_id', 'conservation', 'conservation_ortholog',
       'conservation_species', 'context_detail', 'epigenetic_modification',
       'modification_detail', 'target_type', 'target', 'target_interaction',
       'target_effect', 'biological_process', 'pathway',
       'functional_mechanism', 'description', 'PMID', 'gobp', 'geneid',
       'Source(s)'], sep='\t')
lncRNA_gobp2

In [ ]:
#### disease

lncRNA_gobp2.context_detail = [replace_with_mondo(item) for item in lncRNA_gobp2.context_detail]

#### pathway

lncRNA_gobp2['pathway'] = lncRNA_gobp2['pathway'].str.lower()
lncRNA_gobp2['pathway'] = lncRNA_gobp2['pathway'].str.replace(';', '|')
lncRNA_gobp2['pathway'] = lncRNA_gobp2['pathway'].str.replace('nf-??b', 'nuclear factor kappa B')
lncRNA_gobp2['pathway'] = lncRNA_gobp2['pathway'].str.replace('/??-catenin', '')
lncRNA_gobp2['pathway'] = lncRNA_gobp2['pathway'].str.replace('pi3k/akt ', '')
lncRNA_gobp2['pathway'] = [replace_with_pw(item) for item in lncRNA_gobp2['pathway']]

# lncRNA-pathway

In [ ]:
lncRNA_pw=pd.read_csv(
    edge_data_location + 'testlncRNA-pw.txt', names=['transcript_id','conservation',
                                                     'conservation_ortholog','conservation_species',
                                                     'biological_context','context_detail',
                                                     'tissue/cell line','epigenetic_modification','expression',
                                                     'expression_detail','regulator_type','regulator',
                                                     'regulator_interaction','regulator_effect','target_type',
                                                     'target','target_effect','experimental_method',
                                                     'biological_process','functional_mechanism',
                                                     'clinical_detail','drug','description','pmid','0_y',1,
                                                     'Source(s)'], sep='\t')
lncRNA_pw

In [ ]:
#### disease

lncRNA_pw.context_detail = [replace_with_mondo(item) for item in lncRNA_pw.context_detail]

#### tissue/cell line
lncRNA_pw['tissue/cell line'] = lncRNA_pw['tissue/cell line'].str.lower()
lncRNA_pw['tissue/cell line'] = lncRNA_pw['tissue/cell line'].str.replace(';', '|')
lncRNA_pw['tissue/cell line'] = [replace_with_clo(item) for item in lncRNA_pw['tissue/cell line']]
lncRNA_pw['tissue/cell line'] = [replace_with_clo2(item) for item in lncRNA_pw['tissue/cell line']]
lncRNA_pw['tissue/cell line'] = [replace_with_uberon(item) for item in lncRNA_pw['tissue/cell line']]

#### regulator

lncRNA_pw['regulator'] = lncRNA_pw['regulator'].str.replace(';', '|').str.lower()
lncRNA_pw['regulator'] = [replace_with_pro(item) for item in lncRNA_pw['regulator']]

# target

lncRNA_pw['target'] = lncRNA_pw['target'].str.replace('miR', 'hsa-miR')
lncRNA_pw['target'] = [replace_with_mirbase(item) for item in lncRNA_pw['target']]
lncRNA_pw['target'] = lncRNA_pw['target'].str.replace(';', '|').str.lower()
lncRNA_pw['target'] = [replace_with_pro(item) for item in lncRNA_pw['target']]

#### experimental method
lncRNA_pw['experimental_method'] = lncRNA_pw['experimental_method'].str.replace('wetern', 'western')
lncRNA_pw['experimental_method'] = lncRNA_pw['experimental_method'].str.replace('assay assay', 'assay')
lncRNA_pw['experimental_method'] = lncRNA_pw['experimental_method'].str.lower()
lncRNA_pw['experimental_method'] = lncRNA_pw['experimental_method'].str.replace(';', '|')
lncRNA_pw['experimental_method'] = [replace_with_ncit(item) for item in lncRNA_pw['experimental_method']]

#### biological process

lncRNA_pw['biological_process'] = lncRNA_pw['biological_process'].str.lower()
lncRNA_pw['biological_process'] = lncRNA_pw['biological_process'].str.replace(';', '|')
lncRNA_pw['biological_process'] = [replace_with_go(item) for item in lncRNA_pw['biological_process']]

#### drug

lncRNA_pw['drug'] = lncRNA_pw['drug'].str.lower()
lncRNA_pw['drug'] = [replace_with_drugbank(item) for item in lncRNA_pw['drug']]

# miRNA-TF

In [ ]:
miRNA_TF_up=pd.read_csv(
    edge_data_location + 'testmiRNA-TFup.txt', names=['chrom','strand',
                                                      'Start10kb','End10kb','chromStartTF','chromEndTF',
                                                      'Refseq','score','zScore','1_y',0,'Source(s)'], sep='\t')

miRNA_TF_down=pd.read_csv(
    edge_data_location + 'testmiRNA-TFdown.txt', names=['chrom','strand',
                                                        'Start10kb','End10kb','chromStartTF','chromEndTF',
                                                        'Refseq','score','zScore','1_y',0,'Source(s)'], sep='\t')

miRNA_TF_up

In [ ]:
miRNA_TF_up['chrom'] = miRNA_TF_up['chrom'].str.lower()
miRNA_TF_up['chrom'] = [replace_with_ncit(item) for item in miRNA_TF_up['chrom']]

miRNA_TF_down['chrom'] = miRNA_TF_down['chrom'].str.lower()
miRNA_TF_down['chrom'] = [replace_with_ncit(item) for item in miRNA_TF_down['chrom']]

# premiRNA-TF

In [ ]:
miRNA_TF=pd.read_csv(
    edge_data_location + 'testpremiRNA-TF.txt', names=['action_type', 'pmid', '1_y', 0, 'Source(s)'], sep='\t')
miRNA_TF

#### action type

In [ ]:
miRNA_TF['action_type'] = miRNA_TF['action_type'].str.lower()
miRNA_TF['action_type'] = [replace_with_ncit(item) for item in miRNA_TF['action_type']]
miRNA_TF['action_type'].unique()

# miRNA-chemical

In [ ]:
miRNA_chemical=pd.read_csv(
    edge_data_location + 'testchemical-miRNA.txt', names=['Detection method ','Condition','PMID','Support',1,
                                                          'expression','miRBase Accession',
                                                          'Potential biomarker role defined in the literature',
                                                          'exRNA form','Sample','Sample source',
                                                          'Diseases, Cell Lines or normal status','Methods',
                                                          'Experiment Description/Results','Source(s)'], sep='\t')
miRNA_chemical

#### detection method

In [ ]:
miRNA_chemical['Detection method '] = miRNA_chemical['Detection method '].str.lower()
miRNA_chemical['method'] = miRNA_chemical['method'].str.lower()
miRNA_chemical['Methods'] = miRNA_chemical['Methods'].str.lower()
miRNA_chemical['Detection method '] = miRNA_chemical['Detection method '].str.replace('quantitative real-time pcr', 'http://purl.obolibrary.org/obo/NCIT_C28408')
miRNA_chemical['Detection method '] = miRNA_chemical['Detection method '].str.replace('qrt-pcr', 'http://purl.obolibrary.org/obo/NCIT_C28408')
miRNA_chemical['method'] = miRNA_chemical['method'].str.replace('quantitative real-time pcr', 'http://purl.obolibrary.org/obo/NCIT_C28408')
miRNA_chemical['method'] = miRNA_chemical['method'].str.replace('qrt-pcr', 'http://purl.obolibrary.org/obo/NCIT_C28408')
miRNA_chemical['Methods'] = miRNA_chemical['Methods'].str.replace('real-time rt-pcr', 'http://purl.obolibrary.org/obo/NCIT_C28408')
miRNA_chemical['Methods'] = miRNA_chemical['Methods'].str.replace('qrt-pcr', 'http://purl.obolibrary.org/obo/NCIT_C28408')

miRNA_chemical['Detection method '] = miRNA_chemical['Detection method '].astype(str)
miRNA_chemical['method'] = miRNA_chemical['method'].astype(str)
miRNA_chemical['Detection method '].fillna(miRNA_chemical['method'], inplace=True)
miRNA_chemical.loc[(~miRNA_chemical['Detection method '].isna()) & (~miRNA_chemical['method'].isna()) &
                   (miRNA_chemical['Detection method '] != miRNA_chemical['method']),
                   ["Detection method "]] = miRNA_chemical["Detection method "
                                                         ].astype(str) + '|' + miRNA_chemical['method'].astype(str)

miRNA_chemical['Detection method '] = miRNA_chemical['Detection method '].astype(str)
miRNA_chemical['Methods'] = miRNA_chemical['Methods'].astype(str)
miRNA_chemical['Detection method '].fillna(miRNA_chemical['Methods'], inplace=True)
miRNA_chemical.loc[(~miRNA_chemical['Detection method '].isna()) & (~miRNA_chemical['Methods'].isna()) &
                   (miRNA_chemical['Detection method '] != miRNA_chemical['Methods']),
                   ["Detection method "]] = miRNA_chemical["Detection method "
                                                         ].astype(str) + '|' + miRNA_chemical['Methods'].astype(str)

miRNA_chemical['Detection method '] = [replace_with_ncit(item) for item in miRNA_chemical['Detection method ']]
miRNA_chemical['Detection method '].unique()

#### condition

In [ ]:
miRNA_chemical['Condition'] = miRNA_chemical['Condition'].str.replace(r's$', '', regex=True)
miRNA_chemical['Condition'] = miRNA_chemical['Condition'].str.replace(r's|', '', regex=True)
miRNA_chemical['Condition'] = miRNA_chemical['Condition'].str.replace(', ', '|')
miRNA_chemical['Condition'] = miRNA_chemical['Condition'].str.replace('- ', '-')
miRNA_chemical['Condition'] = miRNA_chemical['Condition'].str.lower()
miRNA_chemical['Condition'] = [replace_with_clo(item) for item in miRNA_chemical['Condition']]
miRNA_chemical['Condition'] = [replace_with_clo2(item) for item in miRNA_chemical['Condition']]
miRNA_chemical['Condition'] = [replace_with_ncit(item) for item in miRNA_chemical['Condition']]
miRNA_chemical['Condition'].unique()

# circRNA-miRNA

In [ ]:
circRNA_miRNA=pd.read_csv(
    edge_data_location + 'testcircRNA-miRNA.txt', names=[2, 6, 9, 10, 11, 12, 13, 14, 15, 16, 17, 'genbank_id',
                                                         '0_y', 'a', 'Source(s)'], sep='\t',dtype=object)
circRNA_miRNA

#### disease

In [ ]:
circRNA_miRNA[16] = circRNA_miRNA[16].str.lower()
circRNA_miRNA[16] = circRNA_miRNA[16].str.replace("[ns]", ']')
circRNA_miRNA[16] = circRNA_miRNA[16].str.replace("]", '|')
circRNA_miRNA[16] = circRNA_miRNA[16].str.replace("[", '')
circRNA_miRNA[16] = circRNA_miRNA[16].str.replace(", ", '|')
circRNA_miRNA[16] = circRNA_miRNA[16].str.replace("||", '|')
circRNA_miRNA[16] = circRNA_miRNA[16].replace("|||", np.nan)
circRNA_miRNA[16] = circRNA_miRNA[16].str.replace('_', ' ')
circRNA_miRNA[16] = circRNA_miRNA[16].str.replace('lung/sclc', 'small cell lung carcinoma')
circRNA_miRNA[16] = circRNA_miRNA[16].replace(r'\(.*?\)', '', regex=True)
circRNA_miRNA[16] = circRNA_miRNA[16].replace(r'\|$', '', regex=True)
circRNA_miRNA[16] = [replace_with_mondo(item) for item in circRNA_miRNA[16]]
print(circRNA_miRNA[16].unique())

In [ ]:
circRNA_miRNA[16].unique()

# RNAInter

In [ ]:
RNA_RNA = pd.read_csv(unprocessed_data_location+'Download_data_RR.tar.gz',sep='\t')
RNA_RNA

In [ ]:
RNA_RNA.strong = RNA_RNA.strong.str.replace('//','|')
RNA_RNA.strong = RNA_RNA.strong.str.lower()
RNA_RNA.strong = [replace_with_ncit(item) for item in RNA_RNA.strong]

RNA_RNA.weak = RNA_RNA.weak.str.replace('//','|')
RNA_RNA.weak = RNA_RNA.weak.str.lower()
RNA_RNA.weak = [replace_with_ncit(item) for item in RNA_RNA.weak]

RNA_RNA.predict = RNA_RNA.predict.str.replace('//','|')
RNA_RNA.predict = RNA_RNA.predict.str.lower()
RNA_RNA.predict = [replace_with_ncit(item) for item in RNA_RNA.predict]

In [ ]:
RNA_protein = pd.read_csv(unprocessed_data_location+'Download_data_RP.tar.gz',sep='\t')
RNA_protein

In [ ]:
RNA_protein.strong = RNA_protein.strong.str.replace('//','|')
RNA_protein.strong = RNA_protein.strong.str.lower()
RNA_protein.strong = [replace_with_ncit(item) for item in RNA_protein.strong]

RNA_protein.weak = RNA_protein.weak.str.replace('//','|')
RNA_protein.weak = RNA_protein.weak.str.lower()
RNA_protein.weak = [replace_with_ncit(item) for item in RNA_protein.weak]

RNA_protein.predict = RNA_protein.predict.str.replace('//','|')
RNA_protein.predict = RNA_protein.predict.str.lower()
RNA_protein.predict = [replace_with_ncit(item) for item in RNA_protein.predict]

In [ ]:
RNA_gene = pd.read_csv(unprocessed_data_location+'Download_data_RD.tar.gz',sep='\t')
RNA_gene

In [ ]:
RNA_gene.strong = RNA_gene.strong.str.replace('//','|')
RNA_gene.strong = RNA_gene.strong.str.lower()
RNA_gene.strong = [replace_with_ncit(item) for item in RNA_gene.strong]

RNA_gene.weak = RNA_gene.weak.str.replace('//','|')
RNA_gene.weak = RNA_gene.weak.str.lower()
RNA_gene.weak = [replace_with_ncit(item) for item in RNA_gene.weak]

RNA_gene.predict = RNA_gene.predict.str.replace('//','|')
RNA_gene.predict = RNA_gene.predict.str.lower()
RNA_gene.predict = [replace_with_ncit(item) for item in RNA_gene.predict]

In [ ]:
RNA_chemical = pd.read_csv(unprocessed_data_location+'Download_data_RC.tar.gz',sep='\t')
RNA_chemical

In [ ]:
RNA_chemical.strong = RNA_chemical.strong.str.replace('//','|')
RNA_chemical.strong = RNA_chemical.strong.str.lower()
RNA_chemical.strong = [replace_with_ncit(item) for item in RNA_chemical.strong]

RNA_chemical.weak = RNA_chemical.weak.str.replace('//','|')
RNA_chemical.weak = RNA_chemical.weak.str.lower()
RNA_chemical.weak = [replace_with_ncit(item) for item in RNA_chemical.weak]

RNA_chemical.predict = RNA_chemical.predict.str.replace('//','|')
RNA_chemical.predict = RNA_chemical.predict.str.lower()
RNA_chemical.predict = [replace_with_ncit(item) for item in RNA_chemical.predict]

In [ ]:
RNA_hisMod = pd.read_csv(unprocessed_data_location+'Download_data_RH.tar.gz',sep='\t')
RNA_hisMod

In [ ]:
RNA_hisMod.strong = RNA_hisMod.strong.str.replace('//','|')
RNA_hisMod.strong = RNA_hisMod.strong.str.lower()
RNA_hisMod.strong = [replace_with_ncit(item) for item in RNA_hisMod.strong]

RNA_hisMod.weak = RNA_hisMod.weak.str.replace('//','|')
RNA_hisMod.weak = RNA_hisMod.weak.str.lower()
RNA_hisMod.weak = [replace_with_ncit(item) for item in RNA_hisMod.weak]

RNA_hisMod.predict = RNA_hisMod.predict.str.replace('//','|')
RNA_hisMod.predict = RNA_hisMod.predict.str.lower()
RNA_hisMod.predict = [replace_with_ncit(item) for item in RNA_hisMod.predict]

# ncRDeathDB

In [ ]:
RNA_pDeath = pd.read_excel(unprocessed_data_location + 'allNcRNACelldeathData.xlsx', dtype={"geneid": "string"})
RNA_pDeath

In [ ]:
print(RNA_pDeath.Action_Mode.unique())
RNA_pDeath.Action_Mode = RNA_pDeath.Action_Mode.str.replace('updown', 'http://purl.obolibrary.org/obo/OMIT_0016489|http://purl.obolibrary.org/obo/OMIT_0016265')
RNA_pDeath.Action_Mode = RNA_pDeath.Action_Mode.str.replace('down', 'http://purl.obolibrary.org/obo/OMIT_0016265')
RNA_pDeath.Action_Mode = RNA_pDeath.Action_Mode.str.replace('dowm', 'http://purl.obolibrary.org/obo/OMIT_0016265')
RNA_pDeath.Action_Mode = RNA_pDeath.Action_Mode.str.replace('up ed', 'http://purl.obolibrary.org/obo/OMIT_0016489')
RNA_pDeath.Action_Mode = RNA_pDeath.Action_Mode.str.replace('up', 'http://purl.obolibrary.org/obo/OMIT_0016489')

In [ ]:
print(RNA_pDeath.Tissue.unique())
RNA_pDeath.Tissue = [replace_with_clo(item) for item in RNA_pDeath.Tissue]
RNA_pDeath.Tissue = [replace_with_clo2(item) for item in RNA_pDeath.Tissue]

# cncRNAdb

In [ ]:
RNA_anatomy = pd.read_excel(unprocessed_data_location + 'Translated ncRNA.xlsx')
RNA_anatomy

In [ ]:
RNA_anatomy['In vivo/vitro assay'] = RNA_anatomy['In vivo/vitro assay'].str.lower()
RNA_anatomy['In vivo/vitro assay'] = [replace_with_ncit(item) for item in RNA_anatomy['In vivo/vitro assay']]

RNA_anatomy['Low-throughput method'] = RNA_anatomy['Low-throughput method'].str.replace(';','|')
RNA_anatomy['Low-throughput method'] = RNA_anatomy['Low-throughput method'].str.lower()
RNA_anatomy['Low-throughput method'] = [replace_with_ncit(item) for item in RNA_anatomy['Low-throughput method']]

RNA_anatomy['High-throuput method'] = RNA_anatomy['High-throuput method'].str.replace(';','|')
RNA_anatomy['High-throuput method'] = RNA_anatomy['High-throuput method'].str.lower()
RNA_anatomy['High-throuput method'] = [replace_with_ncit(item) for item in RNA_anatomy['High-throuput method']]

In [ ]:
RNA_anatomy['In vivo/vitro assay'].unique()

In [ ]:
RNA_anatomy['Tissue/Cell'] = RNA_anatomy['Tissue/Cell'].str.replace(';','|')
RNA_anatomy['Tissue/Cell'] = RNA_anatomy['Tissue/Cell'].str.lower()
RNA_anatomy['Tissue/Cell'] = [replace_with_clo(item) for item in RNA_anatomy['Tissue/Cell']]
RNA_anatomy['Tissue/Cell'] = [replace_with_clo2(item) for item in RNA_anatomy['Tissue/Cell']]
RNA_anatomy['Tissue/Cell'].unique()

In [ ]:
RNA_anatomy = pd.concat([pd.read_excel(unprocessed_data_location + 'Regulatory mRNA.xlsx'),
    pd.read_excel(unprocessed_data_location + 'Scaffold mRNA.xlsx'),
    pd.read_excel(unprocessed_data_location + 'Sponge mRNA.xlsx')])
RNA_anatomy

In [ ]:
RNA_anatomy['In vivo/vitro assay'] = RNA_anatomy['In vivo/vitro assay'].str.lower()
RNA_anatomy['In vivo/vitro assay'] = [replace_with_ncit(item) for item in RNA_anatomy['In vivo/vitro assay']]

RNA_anatomy['Low-throughput method'] = RNA_anatomy['Low-throughput method'].str.replace(';','|')
RNA_anatomy['Low-throughput method'] = RNA_anatomy['Low-throughput method'].str.lower()
RNA_anatomy['Low-throughput method'] = [replace_with_ncit(item) for item in RNA_anatomy['Low-throughput method']]

RNA_anatomy['High-throuput method'] = RNA_anatomy['High-throuput method'].str.replace(';','|')
RNA_anatomy['High-throuput method'] = RNA_anatomy['High-throuput method'].str.lower()
RNA_anatomy['High-throuput method'] = [replace_with_ncit(item) for item in RNA_anatomy['High-throuput method']]

RNA_anatomy['Tissue/Cell'] = RNA_anatomy['Tissue/Cell'].str.replace(';','|')
RNA_anatomy['Tissue/Cell'] = RNA_anatomy['Tissue/Cell'].str.lower()
RNA_anatomy['Tissue/Cell'] = [replace_with_clo(item) for item in RNA_anatomy['Tissue/Cell']]
RNA_anatomy['Tissue/Cell'] = [replace_with_clo2(item) for item in RNA_anatomy['Tissue/Cell']]
RNA_anatomy['Tissue/Cell'].unique()

# Vesiclepedia

In [ ]:
experiments = pd.read_csv('http://microvesicles.org/Archive/VESICLEPEDIA_EXPERIMENT_DETAILS_4.1.txt', sep='\t')
protein_ev = pd.read_csv(unprocessed_data_location+'VESICLEPEDIA_PROTEIN_MRNA_DETAILS_4.1.txt', sep='\t')
protein_ev = pd.merge(protein_ev, experiments, on=['EXPERIMENT ID'])
protein_ev

In [ ]:
protein_ev.SAMPLE = protein_ev.SAMPLE.str.lower()
protein_ev.SAMPLE = protein_ev.SAMPLE.str.replace(';', '|')
protein_ev.SAMPLE = protein_ev.SAMPLE.str.strip()
protein_ev.SAMPLE = [replace_with_clo(item) for item in protein_ev.SAMPLE]
protein_ev.SAMPLE = [replace_with_clo2(item) for item in protein_ev.SAMPLE]
protein_ev.SAMPLE.unique()

In [ ]:
protein_ev['IDENTIFICATIONS'] = protein_ev['IDENTIFICATIONS'].str.replace('Proteins', 'http://purl.obolibrary.org/obo/PR_000000001')
protein_ev['IDENTIFICATIONS'] = protein_ev['IDENTIFICATIONS'].str.replace('Protein ', 'http://purl.obolibrary.org/obo/PR_000000001')
protein_ev['IDENTIFICATIONS'] = protein_ev['IDENTIFICATIONS'].str.replace('Protein', 'http://purl.obolibrary.org/obo/PR_000000001')
protein_ev['IDENTIFICATIONS'] = protein_ev['IDENTIFICATIONS'].str.replace('PTM Ubiquitination', 'http://purl.obolibrary.org/obo/NCIT_C19358')
protein_ev['IDENTIFICATIONS'] = protein_ev['IDENTIFICATIONS'].str.replace('Metabolites', 'http://purl.obolibrary.org/obo/NCIT_C61154')
protein_ev['IDENTIFICATIONS'] = protein_ev['IDENTIFICATIONS'].str.replace('miRNA', 'http://purl.obolibrary.org/obo/SO_0000276')
protein_ev['IDENTIFICATIONS'] = protein_ev['IDENTIFICATIONS'].str.replace('mRNA', 'http://purl.obolibrary.org/obo/SO_0000234')
protein_ev['IDENTIFICATIONS'] = protein_ev['IDENTIFICATIONS'].str.replace('miRNA', 'http://purl.obolibrary.org/obo/SO_0000276')
protein_ev['IDENTIFICATIONS'] = protein_ev['IDENTIFICATIONS'].str.replace('snRNA', 'http://purl.obolibrary.org/obo/SO_0000274')
protein_ev['IDENTIFICATIONS'] = protein_ev['IDENTIFICATIONS'].str.replace('Lipids', 'http://purl.obolibrary.org/obo/CHEBI_18059')
protein_ev['IDENTIFICATIONS'].unique()

In [ ]:
protein_ev['METHODS_y'] = protein_ev['METHODS_y'].str.lower()
protein_ev['METHODS_y'] = [replace_with_ncit(item) for item in protein_ev['METHODS_y']] 
protein_ev.METHODS_y.unique()

In [ ]:
data_downloader('http://purl.obolibrary.org/obo/chmo.owl', ontology_data_location)

chmo_graph = Graph().parse(ontology_data_location + 'chmo.owl')

chmo_label = gets_ontology_class_label(chmo_graph)[0]
chmo_dict = {str(k): {str(i).split('/')[-1] for i in v} for k, v in chmo_label.items()}
list({**chmo_dict}.items())[:5]

with open(unprocessed_data_location + 'DESC_chmo_MAP.txt', 'w') as outfile:
    for k, v in {**chmo_dict}.items():
        outfile.write(str(k) + '\t' + str(v).replace('{','').replace('\'','').replace('}','') + '\n')

desc_chmo_map = pd.read_csv(unprocessed_data_location+'DESC_chmo_MAP.txt', header=None, delimiter='\t')
desc_chmo_map[1] = desc_chmo_map[1].str.split(', ')
desc_chmo_map = desc_chmo_map.explode(1)

chmo_syn = gets_ontology_class_synonym(chmo_graph)[0]
chmo_dict = {str(k): {str(i).split('/')[-1] for i in v} for k, v in chmo_syn.items()}

with open(unprocessed_data_location + 'SYN_chmo_MAP.txt', 'w') as outfile:
    for k, v in {**chmo_dict}.items():
        outfile.write(str(k) + '\t' + str(v).replace('{','').replace('\'','').replace('}','') + '\n')

syn_chmo_map = pd.read_csv(unprocessed_data_location+'SYN_chmo_MAP.txt', header=None, delimiter='\t')
syn_chmo_map[1] = syn_chmo_map[1].str.split(', ')
syn_chmo_map = syn_chmo_map.explode(1)
desc_chmo_map = pd.concat([desc_chmo_map, syn_chmo_map], ignore_index=True)
desc_chmo_map

In [ ]:
chmo_dict = dict(zip(desc_chmo_map[0], 'http://purl.obolibrary.org/obo/'+desc_chmo_map[1]))

def replace_with_chmo(substring):
    if pd.isna(substring):
        return np.nan
    else:
        return '|'.join([chmo_dict.get(part, part) for part in substring.split('|')])

protein_ev['ISOLATION METHOD'] = protein_ev['ISOLATION METHOD'].str.lower()
protein_ev['ISOLATION METHOD'] = [replace_with_chmo(item) for item in protein_ev['ISOLATION METHOD']] 
protein_ev['ISOLATION METHOD'].unique()

In [ ]:
miRNA_ev = pd.read_csv(unprocessed_data_location+'VESICLEPEDIA_MIRNA_DETAILS_4.1.txt', sep='\t')
miRNA_ev = pd.merge(miRNA_ev, experiments, on=['EXPERIMENT ID'])
miRNA_ev

In [ ]:
miRNA_ev.SAMPLE = miRNA_ev.SAMPLE.str.lower()
miRNA_ev.SAMPLE = miRNA_ev.SAMPLE.str.replace(';', '|')
miRNA_ev.SAMPLE = miRNA_ev.SAMPLE.str.replace(r's$', '',regex=True)
miRNA_ev.SAMPLE = miRNA_ev.SAMPLE.str.strip()
miRNA_ev.SAMPLE = [replace_with_clo(item) for item in miRNA_ev.SAMPLE]
miRNA_ev.SAMPLE = [replace_with_clo2(item) for item in miRNA_ev.SAMPLE]
print(miRNA_ev.SAMPLE.unique())

miRNA_ev['IDENTIFICATIONS'] = miRNA_ev['IDENTIFICATIONS'].str.replace('Proteins', 'http://purl.obolibrary.org/obo/PR_000000001')
miRNA_ev['IDENTIFICATIONS'] = miRNA_ev['IDENTIFICATIONS'].str.replace('Protein ', 'http://purl.obolibrary.org/obo/PR_000000001')
miRNA_ev['IDENTIFICATIONS'] = miRNA_ev['IDENTIFICATIONS'].str.replace('Protein', 'http://purl.obolibrary.org/obo/PR_000000001')
miRNA_ev['IDENTIFICATIONS'] = miRNA_ev['IDENTIFICATIONS'].str.replace('PTM Ubiquitination', 'http://purl.obolibrary.org/obo/NCIT_C19358')
miRNA_ev['IDENTIFICATIONS'] = miRNA_ev['IDENTIFICATIONS'].str.replace('Metabolites', 'http://purl.obolibrary.org/obo/NCIT_C61154')
miRNA_ev['IDENTIFICATIONS'] = miRNA_ev['IDENTIFICATIONS'].str.replace('miRNA', 'http://purl.obolibrary.org/obo/SO_0000276')
miRNA_ev['IDENTIFICATIONS'] = miRNA_ev['IDENTIFICATIONS'].str.replace('mRNA', 'http://purl.obolibrary.org/obo/SO_0000234')
miRNA_ev['IDENTIFICATIONS'] = miRNA_ev['IDENTIFICATIONS'].str.replace('miRNA', 'http://purl.obolibrary.org/obo/SO_0000276')
miRNA_ev['IDENTIFICATIONS'] = miRNA_ev['IDENTIFICATIONS'].str.replace('snRNA', 'http://purl.obolibrary.org/obo/SO_0000274')
miRNA_ev['IDENTIFICATIONS'] = miRNA_ev['IDENTIFICATIONS'].str.replace('Lipids', 'http://purl.obolibrary.org/obo/CHEBI_18059')
print(miRNA_ev['IDENTIFICATIONS'].unique())

miRNA_ev['METHODS_y'] = miRNA_ev['METHODS_y'].str.lower()
miRNA_ev['METHODS_y'] = [replace_with_ncit(item) for item in miRNA_ev['METHODS_y']] 
print(miRNA_ev.METHODS_y.unique())

miRNA_ev['ISOLATION METHOD'] = miRNA_ev['ISOLATION METHOD'].str.lower()
miRNA_ev['ISOLATION METHOD'] = [replace_with_chmo(item) for item in miRNA_ev['ISOLATION METHOD']] 
miRNA_ev['ISOLATION METHOD'].unique()

In [ ]:
lipid_ev = pd.read_csv(unprocessed_data_location+'VESICLEPEDIA_LIPID_DETAILS_4.1.txt', sep='\t')
lipid_ev = pd.merge(lipid_ev, experiments, on=['EXPERIMENT ID'])
lipid_ev

In [ ]:
lipid_ev.SAMPLE = lipid_ev.SAMPLE.str.lower()
lipid_ev.SAMPLE = lipid_ev.SAMPLE.str.replace(';', '|')
lipid_ev.SAMPLE = lipid_ev.SAMPLE.str.replace(r's$', '',regex=True)
lipid_ev.SAMPLE = lipid_ev.SAMPLE.str.strip()
lipid_ev.SAMPLE = [replace_with_clo(item) for item in lipid_ev.SAMPLE]
lipid_ev.SAMPLE = [replace_with_clo2(item) for item in lipid_ev.SAMPLE]
print(lipid_ev.SAMPLE.unique())

lipid_ev['IDENTIFICATIONS'] = lipid_ev['IDENTIFICATIONS'].str.replace('Proteins', 'http://purl.obolibrary.org/obo/PR_000000001')
lipid_ev['IDENTIFICATIONS'] = lipid_ev['IDENTIFICATIONS'].str.replace('Protein ', 'http://purl.obolibrary.org/obo/PR_000000001')
lipid_ev['IDENTIFICATIONS'] = lipid_ev['IDENTIFICATIONS'].str.replace('Protein', 'http://purl.obolibrary.org/obo/PR_000000001')
lipid_ev['IDENTIFICATIONS'] = lipid_ev['IDENTIFICATIONS'].str.replace('PTM Ubiquitination', 'http://purl.obolibrary.org/obo/NCIT_C19358')
lipid_ev['IDENTIFICATIONS'] = lipid_ev['IDENTIFICATIONS'].str.replace('Metabolites', 'http://purl.obolibrary.org/obo/NCIT_C61154')
lipid_ev['IDENTIFICATIONS'] = lipid_ev['IDENTIFICATIONS'].str.replace('miRNA', 'http://purl.obolibrary.org/obo/SO_0000276')
lipid_ev['IDENTIFICATIONS'] = lipid_ev['IDENTIFICATIONS'].str.replace('mRNA', 'http://purl.obolibrary.org/obo/SO_0000234')
lipid_ev['IDENTIFICATIONS'] = lipid_ev['IDENTIFICATIONS'].str.replace('miRNA', 'http://purl.obolibrary.org/obo/SO_0000276')
lipid_ev['IDENTIFICATIONS'] = lipid_ev['IDENTIFICATIONS'].str.replace('snRNA', 'http://purl.obolibrary.org/obo/SO_0000274')
lipid_ev['IDENTIFICATIONS'] = lipid_ev['IDENTIFICATIONS'].str.replace('Lipids', 'http://purl.obolibrary.org/obo/CHEBI_18059')
print(lipid_ev['IDENTIFICATIONS'].unique())

lipid_ev['METHODS_y'] = lipid_ev['METHODS_y'].str.lower()
lipid_ev['METHODS_y'] = [replace_with_ncit(item) for item in lipid_ev['METHODS_y']] 
print(lipid_ev.METHODS_y.unique())

lipid_ev['ISOLATION METHOD'] = lipid_ev['ISOLATION METHOD'].str.lower()
lipid_ev['ISOLATION METHOD'] = [replace_with_chmo(item) for item in lipid_ev['ISOLATION METHOD']] 
lipid_ev['ISOLATION METHOD'].unique()

# DirectRMDB

In [ ]:
DirectRMDB = pd.read_csv(unprocessed_data_location+'DirectRMDB.txt', sep='\t')
DirectRMDB2 = pd.read_csv(unprocessed_data_location+'HomoSapiens_miRNA.txt', sep='\t')
DirectRMDB = pd.merge(DirectRMDB, DirectRMDB2,on ='ID')
DirectRMDB

In [ ]:
DirectRMDB.software = DirectRMDB.software.str.replace(';','|')
DirectRMDB.software.unique()

In [ ]:
DirectRMDB.cell_line = DirectRMDB.cell_line.str.replace(';','|')
DirectRMDB.cell_line = DirectRMDB.cell_line.str.lower()
DirectRMDB.cell_line = DirectRMDB.cell_line.str.replace(';', '|')
DirectRMDB.cell_line = [replace_with_clo(item) for item in DirectRMDB.cell_line]
DirectRMDB.cell_line = [replace_with_clo2(item) for item in DirectRMDB.cell_line]
DirectRMDB.cell_line.unique()

# Modomics

In [ ]:
modomics = pd.read_csv(unprocessed_data_location+'modomics.csv')
modomics

In [ ]:
modomics['Disease Name'] = modomics['Disease Name'].str.lower()
modomics['Disease Name'] = [replace_with_mondo(item) for item in modomics['Disease Name']]
modomics['Disease Name'].unique()